In [1]:
%cd ..

/Users/shivamkaushik/Code/ik-agent


In [2]:
from src.modules.ik_transforms.transformation_builder import (
    load_transform_metadata,
    small_llm_filter,
    filter_metadata,
    VALID_METHODS_DOCS_STRINGS,
    big_llm_generate,
    TransformPlan,
    search_docs,
    group_search_results,
    extract_params_from_docs,
    resolve_imagekit_transform
)

Deprecation warning: AnalyticsRulesV1 is deprecated on v30+. Use client.analytics instead.


In [3]:
df = load_transform_metadata(csv_path="static/ik_transforms.csv")
df

,parameter_name,ik_key,value_type,allowed_values,constraints,tags,used_in,description,example_usage,method_name
0,width,width,"number, arithmetic_expression, """"""auto""""""",integer > 1 (px) OR 0 < w < 1 (percentage) OR ...,Auto width from Client Hint Sec-CH-Width.,"resize, crop",image,"Output width. If only w is provided, height au...",### Example 1\n\n**Integer value**\n\n```pytho...,resize_and_crop
1,height,height,"number, arithmetic_expression, """"""auto""""""",integer > 1 (px) OR 0 < w < 1 (percentage) OR ...,NaN,resize,image,"Output height. If only h is provided, width au...",### Example 1\n\n**Integer value**\n\n```pytho...,resize_and_crop
2,Aspect Ratio,aspect_ratio,"<w>-<h>, arithmetic_expression","width-height, arithmetic expression",Must be used with either w or h. Ignored if bo...,resize,image,Aspect ratio (width:height).,"### Example 1\n\n**width = 400, height = 300, ...",resize_and_crop
3,Crop Mode,crop_mode,enum,pad_resize | pad_extract | extract,pad_resize\n\n* Requires **both `w` and `h`** ...,"resize, crop",image,`cm-pad_resize` resizes the image to fit withi...,## pad_resize\n\n### Example: Equal padding on...,resize_and_crop
4,Crop,crop,enum,force | at_max_enlarge | at_least | maintain_r...,## force (`force`)\n\n* Requires **both `w` an...,crop,image,**force (`force`)**\nResizes the image to the ...,---\n\n## force (`force`)\n\n### This example ...,resize_and_crop
5,Focus Custom,focus,"""""""custom""""""","""custom""",* Requires a **custom focus area to be defined...,crop,image,Uses a **predefined custom focus area** (set d...,---\n\n## custom with maintain_ratio\n\n### Th...,resize_and_crop
6,Focus Auto - Smart Crop,focus,"""""""auto""""""","""auto""",* Requires **both `w` and `h`**\n* Works only ...,"crop, smart_crop",image,Automatically detects the most important regio...,\n## fo-auto with default crop strategy (maint...,resize_and_crop
7,Focus Face - Smart Crop,focus,"""""""face""""""","""face""",\n* Requires **both `w` and `h`**\n* Works onl...,"crop, smart_crop",image,Automatically detects one or more faces in the...,## fo-auto with default crop strategy (maintai...,resize_and_crop
8,Focus Objects - Smart Crop,focus,"""""""object""""""","""object"" | ""<object1>_<object2>""",\n* Object name must be from the **supported C...,"crop, smart_crop",image,Automatically detects a specified object in th...,## Zoom with fo-object and extract crop mode\n...,resize_and_crop
9,Zoom,zoom,number,"0-1, >1\n",* Works **only with focus-based cropping**\n\n...,crop,image,Controls how much to zoom in or zoom out aroun...,"\n## Zoom with tightly cropped object, aspect ...",resize_and_crop


In [4]:
import yaml

with open("static/transforms_method_capabilities.yaml", "r") as f:
    method_n_capabilities = yaml.safe_load(f)

In [5]:
 df[["method_name", "tags"]].to_dict(orient="records")

[{'method_name': 'resize_and_crop', 'tags': 'resize, crop'},
 {'method_name': 'resize_and_crop', 'tags': 'resize'},
 {'method_name': 'resize_and_crop', 'tags': 'resize'},
 {'method_name': 'resize_and_crop', 'tags': 'resize, crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop, smart_crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop, smart_crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop, smart_crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop'},
 {'method_name': 'resize_and_crop', 'tags': 'crop'},
 {'method_name': 'resize_and_crop', 'tags': 'resize'}]

In [6]:
method_n_capabilities

{'resize_and_crop': {'capability': ['resize by width',
   'resize by height',
   'responsive resize auto dimension',
   'arithmetic expression based resize',
   'aspect ratio based resize',
   'aspect ratio enforced crop',
   'crop force resize',
   'crop at least resize',
   'crop at max enlarge resize',
   'crop maintain ratio resize',
   'extract crop',
   'padded extract crop',
   'padded resize',
   'add padding while resizing an image to specific dimensions',
   'intelligent focus auto',
   'intelligent focus face detection',
   'intelligent focus object detection',
   'positional focus padding control',
   'positional focus relative crop',
   'custom focus region',
   'zoom based cropping',
   'absolute coordinate crop',
   'center based coordinate crop',
   'device pixel ratio scaling']}}

In [7]:
from src.config import (
    IK_TRANSFORMS_CSV_PATH,
)

user_query = "resize image and add padding. resize to 800x600, add image overlay"

df = load_transform_metadata(IK_TRANSFORMS_CSV_PATH)

classification = await small_llm_filter(
    user_query=user_query,
    valid_methods=method_n_capabilities,
)

In [8]:
from toon_format import encode

print(encode(method_n_capabilities))

resize_and_crop:
  capability[24]: resize by width,resize by height,responsive resize auto dimension,arithmetic expression based resize,aspect ratio based resize,aspect ratio enforced crop,crop force resize,crop at least resize,crop at max enlarge resize,crop maintain ratio resize,extract crop,padded extract crop,padded resize,add padding while resizing an image to specific dimensions,intelligent focus auto,intelligent focus face detection,intelligent focus object detection,positional focus padding control,positional focus relative crop,custom focus region,zoom based cropping,absolute coordinate crop,center based coordinate crop,device pixel ratio scaling


In [9]:
classified_methods = classification.get("methods", [])
unresolved_intent = classification.get("unresolved_intent")

In [10]:
filtered_metadata = filter_metadata(
    df=df,
    methods=classified_methods,
)

output_metadata = []
for method_meta_data in filtered_metadata:
    method_name = method_meta_data.get("method_name")
    output_metadata.append(
        {
            "method": method_name,
            "method_description": VALID_METHODS_DOCS_STRINGS.get(method_name, ""),
            "method_metadata": method_meta_data,
        }
    )

structured_plan: TransformPlan = []

if output_metadata:
    structured_plan = await big_llm_generate(
        user_query=user_query,
        filtered_metadata=output_metadata,
    )

In [11]:
structured_plan

[{'method': 'resize_and_crop',
  'params': {'width': 800, 'height': 600, 'crop_mode': 'pad_resize'}}]

In [12]:
doc_params = {}
should_query_docs = not structured_plan or bool(unresolved_intent)

if should_query_docs:
    search_query = unresolved_intent or user_query
    # search_query = 'how to add image overlay to a picture,'
    raw_results = await search_docs(query=search_query)
    grouped = group_search_results(raw_results)
    doc_params = await extract_params_from_docs(
        user_query=user_query,
        # user_query='how to add logo @ bottom right corner to an image',
        search_docs_result=grouped,
    )

In [13]:
grouped

{'https://imagekit.io/docs/add-overlays-on-images': {'page_title': 'Add overlays on images',
  'page_description': 'A comprehensive guide on adding images and text on top of a base image using ImageKit.',
  'content': '\n## Apply transformation on image overlay > List of supported image transformations in image layers\n**Summary:** This section provides a table of all supported transformation parameters for image layers—covering size (`w`, `h`, `ar`), crop (`c`, `cm`, `fo`, `z`), styling (`b`, `o`, `bg`, `r`, `rt`, `fl`, `t`), and output controls (`dpr`, `q`)—including where they are documented and how some accept arithmetic expressions. It matters because it defines exactly which transformations can be applied to overlays and how to configure them, including special behaviors like trimming thresholds and background opacity encoding.\n\n| Parameter                                                                                          | Description                                     

In [14]:
structured_plan

[{'method': 'resize_and_crop',
  'params': {'width': 800, 'height': 600, 'crop_mode': 'pad_resize'}}]

In [15]:
from src.modules.ik_transforms.transforms.resize_n_crop import ResizeAndCropTransforms

def parse_params(
    method: str,
    params: dict,
):
    """Parse and normalize parameters based on method capabilities."""
    if method == "resize_and_crop":
        return ResizeAndCropTransforms().resize_and_crop(**params)
    return params

In [16]:
for step in structured_plan:
    method = step.get("method", "")
    params = parse_params(method, step.get("params", {}))

In [17]:
parse_params(method, params)

{'width': '800',
 'height': '600',
 'crop': 'maintain_ratio',
 'crop_mode': 'pad_resize'}

In [18]:
base_transform = {}
overlay_transforms = []

# Overlay steps remain isolated
if "overlay" in method:
    overlay_transforms.append({k: v for k, v in params.items()})
else:
    # Merge base (non-overlay) params
    for k, v in params.items():
        base_transform[k] = v

In [19]:
base_transform

{'width': '800',
 'height': '600',
 'crop': 'maintain_ratio',
 'crop_mode': 'pad_resize'}

In [20]:
from src.utils.utils import get_transform_key
# Apply doc params ONLY if not already set
if doc_params and doc_params.get("params"):
    for long_key, value in doc_params["params"].items():
        short_key = get_transform_key(long_key)
        if short_key not in base_transform:
            base_transform[short_key] = value

In [21]:
base_transform

{'width': '800',
 'height': '600',
 'crop': 'maintain_ratio',
 'crop_mode': 'pad_resize'}

In [22]:
overlay_transforms

[]

In [3]:
user_query = "resize image and add padding. resize to 800x600, add image overlay"
result = await resolve_imagekit_transform(
    user_query=user_query,
)
result

[{'width': '800',
  'height': '600',
  'crop': 'maintain_ratio',
  'crop_mode': 'pad_resize'}]